In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

np.random.seed(11)
t = np.linspace(0, 4 * np.pi, 120)
x = np.sin(t) + 0.3 * np.sin(3 * t) + 0.2 * np.random.randn(len(t))

# Sample random intervals
n_intervals = 6
intervals = []
for _ in range(n_intervals):
    start = np.random.randint(0, len(x) - 10)
    end = np.random.randint(start + 5, min(start + 30, len(x)))
    intervals.append((start, end))

fig = go.Figure()
fig.add_trace(go.Scatter(y=x, mode='lines', name='series'))
for start, end in intervals:
    fig.add_vrect(x0=start, x1=end, fillcolor='orange', opacity=0.2, line_width=0)
fig.update_layout(title='Random intervals used for feature extraction', xaxis_title='time index')
fig.show()

rows = []
for idx, (start, end) in enumerate(intervals, start=1):
    segment = x[start:end]
    slope = np.polyfit(np.arange(len(segment)), segment, 1)[0]
    rows.append({
        'interval': f'I{idx} [{start},{end})',
        'mean': segment.mean(),
        'std': segment.std(ddof=0),
        'slope': slope,
        'min': segment.min(),
        'max': segment.max(),
    })

features_df = pd.DataFrame(rows)
features_df


,interval,mean,std,slope,min,max
0,"I1 [57,84)",0.650330,0.462681,0.037496,-0.643084,1.257621
1,"I2 [35,41)",-0.839211,0.198921,-0.012329,-1.225615,-0.632476
2,"I3 [107,114)",-0.787726,0.371754,-0.114739,-1.322912,-0.376118
3,"I4 [43,64)",-0.492651,0.478912,0.060491,-1.129733,0.615441
4,"I5 [0,6)",0.373543,0.264580,0.095520,0.024957,0.739853
5,"I6 [87,116)",-0.630118,0.401690,-0.026529,-1.322912,0.597618


In [2]:
try:
    import pandas as pd
    from sktime.registry import all_estimators

    def _estimators_df():
        try:
            return all_estimators(estimator_types='classifier', as_dataframe=True)
        except TypeError:
            ests = all_estimators(estimator_types='classifier')
            return pd.DataFrame(
                [{'name': n, 'class': c.__name__, 'module': c.__module__} for n, c in ests]
            )

    df = _estimators_df()
    mask = df['module'].str.contains('interval', case=False, na=False)
    df_interval = df[mask].sort_values('name')
    df_interval
except Exception as exc:
    print('sktime not installed or registry unavailable:', exc)


sktime not installed or registry unavailable: No module named 'sktime'


In [3]:
try:
    from sktime.datasets import load_basic_motions
    from sktime.classification.interval_based import TimeSeriesForestClassifier
    from sklearn.metrics import classification_report

    X_train, y_train = load_basic_motions(split='train', return_X_y=True)
    X_test, y_test = load_basic_motions(split='test', return_X_y=True)

    clf = TimeSeriesForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(classification_report(y_test, pred))
except Exception as exc:
    print('Install sktime to run the interval-based demo:', exc)


Install sktime to run the interval-based demo: No module named 'sktime'
